# Convert numbers to fixed point

In [1]:
import numpy as np

def add_1(complement):
    n = len(complement)
    carry = 1
    i = n - 1
    complement = list(complement)

    while i > 0 and carry:
        if complement[i] == '0' and carry == 1:
            complement[i] = '1'
            carry = 0
        if complement[i] == '1' and carry == 1:
            complement[i] = '0'
        i -= 1
        #print(carry)

    return "".join(complement)
    
def decimal_dec_to_bin(x, decimal_digit):
    """
    :param x: is the original number
    :param decimal_digit: the number of digits of decimal part.
    :return:
    """
    x = abs(x)
    x -= int(x)
    bins = ""
    i = 0
    while x and i < decimal_digit:
        x *= 2
        bins += "1" if x >= 1. else "0"
        x -= int(x)
        i += 1
    return bins

def dec_to_bin(x, int_digit, decimal_digit, decimal_point=False):
    """
    :param x: original number
    :param int_digit: the number of digits of integer part.
    :param decimal_digit: the number of digits of decimal part.
    :return:
    """
    decimal = decimal_dec_to_bin(x, decimal_digit)
    if x >= 0:  # true form = complement
        integer = bin(int(x))[2:]
        if integer == "0":
            integer = ""
        add_zero_int = int_digit - len(integer)
        add_zero_integer_complement ="0" + "0" * add_zero_int + integer
    if x < 0:
        integer = bin(int(x))[3:]
        if integer == "0":
            integer = ""
        add_zero_int = int_digit - len(integer)
        add_zero_integer_true = "1" + "0" * add_zero_int + integer # true form
        
        add_zero_integer_reverse = "1" + "".join(str(0 if int(r) == 1 else 1) for r in add_zero_integer_true[1:]) # radix-minus-one complement
        add_zero_integer_complement = add_1(add_zero_integer_reverse)
        #print(add_zero_integer_complement)

         # radix-minus-one complement
    add_zero_dec = decimal_digit - len(decimal)
    add_zero_dec = decimal + "0" * add_zero_dec
    
    if decimal_point:
        return add_zero_integer_complement + "." + add_zero_dec
    else:
        return add_zero_integer_complement + add_zero_dec

def dec_to_bin_array(x_arr, int_digit, decimal_digit, decimal_point=False):
    """
    :param x_arr: an numpy nd array of number
    :param int_digit: the number of digits of integer part.
    :param decimal_digit: the number of digits of decimal part.
    
    return: flatten list of corresponding fixed points
    """
    fixed_point = []
    x_flatten = np.ndarray.flatten(x_arr)
    for i in range(x_flatten.shape[0]):
        fixed_point.append(dec_to_bin(x_flatten[i], int_digit, decimal_digit, decimal_point))
        
    return fixed_point
        
    

In [2]:
#question, int_digit + decimal_digit == 15 and 1 bit for sign right?


# now, why two zeros before '.'? should be only 1
dec_to_bin(0.5, int_digit=0, decimal_digit=15)

'0100000000000000'

In [3]:
dec_to_bin(1.5, int_digit=0, decimal_digit=15)

'01100000000000000'

In [4]:
# set int_digit = 1 and same fraction part as before (just 1 less zero), this time we have a correct answer
dec_to_bin(0.5, int_digit=1, decimal_digit=14)

'0010000000000000'

In [5]:
# things becomes normal
dec_to_bin(0.5, int_digit=2, decimal_digit=13)

'0001000000000000'

In [6]:
# now, disaster appears
dec_to_bin(-0.5, int_digit=0, decimal_digit=15)

'1100000000000000'

In [7]:
# disaster appears again
dec_to_bin(-0.5, int_digit=1, decimal_digit=15)

'10100000000000000'

In [8]:
# disaster appears again and again :(
dec_to_bin(-0.5, int_digit=2, decimal_digit=15)

'100100000000000000'

In [9]:
print("1.25\t:\t", dec_to_bin(1.25, int_digit=5, decimal_digit=5))
print("-1.25\t:\t", dec_to_bin(-1.25, int_digit=5, decimal_digit=5))
print("0.875\t:\t", dec_to_bin(0.875, int_digit=5, decimal_digit=5))
print("13.212\t:\t", dec_to_bin(13.21234, int_digit=5, decimal_digit=5))

1.25	:	 00000101000
-1.25	:	 11111101000
0.875	:	 00000011100
13.212	:	 00110100110


In [10]:
x = np.array([[1.25, -1.25], [0.875, 13.212]])
print(dec_to_bin_array(x, 7, 8, decimal_point=True))
print(dec_to_bin_array(x, 7, 8, decimal_point=False))

['00000001.01000000', '11111111.01000000', '00000000.11100000', '00001101.00110110']
['0000000101000000', '1111111101000000', '0000000011100000', '0000110100110110']


# Convert Image to 0/1 txt

In [45]:
from PIL import Image
import numpy as np

def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

def write_img_bin(img_dir='../img/resize_img/1.bmp', out_filename='img.txt'):
    """
    for image, fixed 8 bits integer, 7 bits decimal
    write the image to a txt file of 0 and ...
    """
    img = load_image(img_dir)
    assert img.shape == (224, 224, 3)
    
    bin_img = dec_to_bin_array(img,8,7,decimal_point=False)
    img_str = "".join(bin_img)
    assert len(img_str) / 224 / 224 / 16 / 3 == 1
    
    with open(out_filename, 'w') as f:
        f.write("%s" % img_str)

In [46]:
write_img_bin('../img/resize_img/{}.bmp'.format(1), 'img.txt')

In [40]:
# save image to txt file
img_str = "".join(bin_img)
type(img_str)




In [42]:
print(len(img_str))
len(img_str) / 224 / 224 / 16 / 3

2408448


1.0